

<h1><center>Decision Trees + Ensemble + Random Forest</center></h1>

In this lab exercise, you will learn a popular machine learning algorithm, Decision Tree. You will use this classification algorithm to build a model from historical data of patients, and their response to different medications. Then you use the trained decision tree to predict the class of a unknown patient, or to find a proper drug for a new patient.

Design classification decision tree from scratch 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

In [0]:
my_data = pd.read_csv("/content/drive/My Drive/drug200.csv", delimiter=",")
my_data[0:5]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


Using my_data as the Drug.csv data read by pandas, declare the following variables:

- X as the Feature Matrix (data of my_data)
- y as the response vector (target)

Remove the column containing the target name since it doesn't contain numeric values.

In [0]:
train = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K', 'Drug']].values
train[0:200]

array([[23, 'F', 'HIGH', 'HIGH', 25.355, 'drugY'],
       [47, 'M', 'LOW', 'HIGH', 13.093, 'drugC'],
       [47, 'M', 'LOW', 'HIGH', 10.113999999999999, 'drugC'],
       ...,
       [52, 'M', 'NORMAL', 'HIGH', 9.894, 'drugX'],
       [23, 'M', 'NORMAL', 'NORMAL', 14.02, 'drugX'],
       [40, 'F', 'LOW', 'NORMAL', 11.349, 'drugX']], dtype=object)

As you may figure out, some features in this dataset are categorical such as Sex or BP. We need to make new dummy variable to hold up the categorical data

In [0]:
len(train)

200

In [0]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
train[:,1] = le_sex.transform(train[:,1]) 

le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
train[:,2] = le_BP.transform(train[:,2])

le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
train[:,3] = le_Chol.transform(train[:,3])

le_drug = preprocessing.LabelEncoder()
le_drug.fit([ 'drugA', 'drugB','drugC', 'drugX', 'drugY']) 
train[:,5] = le_drug.transform(train[:,5])


Modelling with tress with Gini


The key to the CART algorithm is finding the optimal feature and threshold such that the Gini impurity is minimized. To do so, we try all possible splits and compute the resulting Gini impurities.

But how can we try all possible thresholds for a continuous values? There is a simple trick — sort the values for a given feature, and consider all midpoints between two adjacent values. Sorting is costly, but it is needed anyway as we will see shortly.
Now, how might we compute the Gini of all possible splits?

The first solution is to actually perform each split and compute the resulting Gini. Unfortunately this is slow, since we would need to look at all the samples to partition them into left and right. More precisely, it would be n splits with O(n) operations for each split, making the overall operation O(n²).


Refer to https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775

how to calculate the Gini index 

In [0]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

#Splitting a dataset means separating a dataset into two lists of rows given 
#the index of an attribute and a split value for that attribute.
#Once we have the two groups, we can then use our Gini score above to 
#evaluate the cost of the split.
#Splitting a dataset involves iterating over each row, 
#checking if the attribute value is below or above the split value and assigning 
#it to the left or right group respectively.

def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right
 
#Given a dataset, we must check every value on each attribute as a candidate split, 
# evaluate the cost of the split and find the best possible split we could make.
# Once the best split is found, we can use it as a node in our decision tree.
# This is an exhaustive and greedy algorithm.

def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			print('Var%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [0]:

split = get_split(train)
print('Split: [Variable%d < %.3f]' % ((split['index']+1), split['value']))

Var1 < 26.000 Gini=0.673
Var1 < 41.000 Gini=0.668
Var1 < 28.000 Gini=0.672
Var1 < 19.000 Gini=0.670
Var1 < 50.000 Gini=0.659
Var1 < 24.000 Gini=0.675
Var1 < 72.000 Gini=0.674
Var1 < 74.000 Gini=0.675
Var1 < 37.000 Gini=0.669
Var1 < 31.000 Gini=0.671
Var1 < 22.000 Gini=0.671
Var1 < 20.000 Gini=0.671
Var1 < 28.000 Gini=0.672
Var1 < 59.000 Gini=0.662
Var1 < 15.000 Gini=0.676
Var1 < 51.000 Gini=0.656
Var1 < 45.000 Gini=0.664
Var1 < 33.000 Gini=0.671
Var1 < 39.000 Gini=0.669
Var1 < 29.000 Gini=0.671
Var1 < 60.000 Gini=0.665
Var1 < 24.000 Gini=0.675
Var1 < 49.000 Gini=0.663
Var1 < 37.000 Gini=0.669
Var1 < 32.000 Gini=0.670
Var1 < 21.000 Gini=0.673
Var1 < 23.000 Gini=0.674
Var1 < 40.000 Gini=0.668
Var1 < 38.000 Gini=0.670
Var1 < 47.000 Gini=0.664
Var1 < 22.000 Gini=0.671
Var1 < 47.000 Gini=0.664
Var1 < 30.000 Gini=0.670
Var1 < 69.000 Gini=0.673
Var1 < 42.000 Gini=0.667
Var1 < 45.000 Gini=0.664
Var1 < 49.000 Gini=0.663
Var1 < 72.000 Gini=0.674
Var1 < 74.000 Gini=0.675
Var1 < 66.000 Gini=0.668


we now know that age <51 is the best split
Similarily, Female and Male is 0.676


Now that we know how to find the best split points in a dataset

Let's see how we can use it to build a decision tree


In [0]:
# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)
 
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root
 
# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

In [0]:
tree = build_tree(train, 5,30)
print_tree(tree)

In [0]:
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.3, random_state=3)

In [0]:
len(test)

42

In [0]:
def decision_tree(train, test, max_depth, min_size):
	tree = build_tree(train, max_depth, min_size)
	predictions = list()
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)
 

In [0]:
decision_tree(train, test, 5, 30)

Building Random Forest

Building the forest just means building multiple trees, so we call the build_tree() function multiple times with bootstrapped training samples

In [0]:
def build_forest(train, k, N_trees):
    max_depth = 5
    min_size = 30
    forest = []
    for i in range(0, N_trees):
        # bootstrap training dataset
        k_indices = np.random.choice(len(train), k)
        forest.append(build_tree(train[k_indices], max_depth, min_size))
    return forest

def traverse_tree(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return traverse_tree(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return traverse_tree(node['right'], row)
        else:
            return node['right']

def make_prediction(forest, row):
    list_of_classes = []
    for tree_root in forest:
        list_of_classes.append(traverse_tree(tree_root, row))
    return max(set(list_of_classes), key=list_of_classes.count)

In [0]:
forest = build_forest(train, 10, 100)
for row in train:
  predictioin = make_prediction(forest, row)
  print('truth = %d : prediction = %d' % (row[-1], prediction))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Var5 < 14.216 Gini=0.567
Var1 < 50.000 Gini=0.448
Var1 < 68.000 Gini=0.578
Var1 < 60.000 Gini=0.450
Var1 < 19.000 Gini=0.680
Var1 < 20.000 Gini=0.644
Var1 < 41.000 Gini=0.550
Var1 < 39.000 Gini=0.600
Var1 < 35.000 Gini=0.583
Var1 < 28.000 Gini=0.543
Var1 < 26.000 Gini=0.600
Var2 < 1.000 Gini=0.583
Var2 < 1.000 Gini=0.583
Var2 < 1.000 Gini=0.583
Var2 < 0.000 Gini=0.680
Var2 < 1.000 Gini=0.583
Var2 < 1.000 Gini=0.583
Var2 < 0.000 Gini=0.680
Var2 < 1.000 Gini=0.583
Var2 < 0.000 Gini=0.680
Var2 < 0.000 Gini=0.680
Var3 < 0.000 Gini=0.680
Var3 < 0.000 Gini=0.680
Var3 < 0.000 Gini=0.680
Var3 < 0.000 Gini=0.680
Var3 < 0.000 Gini=0.680
Var3 < 0.000 Gini=0.680
Var3 < 1.000 Gini=0.600
Var3 < 2.000 Gini=0.556
Var3 < 1.000 Gini=0.600
Var3 < 0.000 Gini=0.680
Var4 < 0.000 Gini=0.680
Var4 < 0.000 Gini=0.680
Var4 < 1.000 Gini=0.514
Var4 < 1.000 Gini=0.514
Var4 < 1.000 Gini=0.514
Var4 < 1.000 Gini=0.514
Var4 < 1.000 Gini=0.514
Var4 < 1.000 Gini=0.514
Var4 < 0.000 Gini

In [0]:
train_X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
train_y = my_data[['Drug']].values


In [0]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
train_X[:,1] = le_sex.transform(train_X[:,1]) 

le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
train_X[:,2] = le_BP.transform(train_X[:,2])

le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
train_X[:,3] = le_Chol.transform(train_X[:,3])


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

forest = RandomForestClassifier(n_estimators=100)
forest.fit(train_X, train_y)

forest.predict(train_X)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


array(['drugY', 'drugC', 'drugC', 'drugX', 'drugY', 'drugX', 'drugY',
       'drugC', 'drugY', 'drugY', 'drugC', 'drugY', 'drugY', 'drugY',
       'drugX', 'drugY', 'drugX', 'drugA', 'drugC', 'drugY', 'drugY',
       'drugY', 'drugY', 'drugY', 'drugY', 'drugY', 'drugY', 'drugX',
       'drugY', 'drugY', 'drugX', 'drugB', 'drugX', 'drugY', 'drugX',
       'drugX', 'drugA', 'drugX', 'drugX', 'drugX', 'drugY', 'drugB',
       'drugY', 'drugX', 'drugX', 'drugX', 'drugA', 'drugC', 'drugY',
       'drugY', 'drugY', 'drugX', 'drugY', 'drugY', 'drugB', 'drugC',
       'drugB', 'drugY', 'drugX', 'drugY', 'drugY', 'drugA', 'drugY',
       'drugX', 'drugB', 'drugY', 'drugA', 'drugX', 'drugY', 'drugY',
       'drugB', 'drugY', 'drugX', 'drugY', 'drugY', 'drugY', 'drugA',
       'drugY', 'drugA', 'drugX', 'drugB', 'drugX', 'drugC', 'drugA',
       'drugC', 'drugB', 'drugX', 'drugY', 'drugY', 'drugY', 'drugY',
       'drugY', 'drugY', 'drugY', 'drugY', 'drugX', 'drugY', 'drugY',
       'drugY', 'dru